<a href="https://colab.research.google.com/github/pawankulkarni87/Textsummarizer/blob/main/textsummarization_using_BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [2]:
from datasets import load_dataset

ds = load_dataset("sujayC66/text_summarization_512_length_1_4000")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/480 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.67M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/642k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3377 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/597 [00:00<?, ? examples/s]

In [5]:
import pandas as pd


In [6]:
!pip install transformers

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [19]:
#Tokenization
#Tokenization
def preprocess_function(batch):
  source=batch['content']
  target=batch['summary']
  source_id=tokenizer(source,  padding='max_length',max_length=128, truncation=True, return_tensors="pt") # Added truncation and return_tensors
  target_id=tokenizer(target,  padding='max_length',max_length=128, truncation=True, return_tensors="pt") # Added truncation and return_tensors

  labels=target_id['input_ids'] # Changed 'input_id' to 'input_ids'
  labels=[[(label if label != tokenizer.pad_token_id else -100) for label in labels_example] for labels_example in labels.tolist()] # Convert labels to list

  return{
      "input_ids":source_id["input_ids"].tolist(), # Changed 'input_id' to 'input_ids' and convert to list
      "attention_mask":source_id["attention_mask"].tolist(), # Convert to list
      "labels":labels
      }



    # Setup the tokenizer for targets



    # Setup the tokenizer for targets

In [20]:
df_source=ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/3377 [00:00<?, ? examples/s]

Map:   0%|          | 0/597 [00:00<?, ? examples/s]

In [35]:
#training arguments
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# Training arguments, unchanged
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    predict_with_generate=True, # Added for text generation tasks
)

# Instantiate the Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=df_source['train'],
    eval_dataset=df_source['test']
)


In [38]:
trainer.train_dataset

Dataset({
    features: ['content', 'summary', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3377
})

In [39]:
test=trainer.eval_dataset

In [40]:
test

Dataset({
    features: ['content', 'summary', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 597
})

In [41]:
#Save Model

In [48]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

model.save_pretrained('/results/model_directory')
tokenizer.save_pretrained('/results/model_directory')

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('/results/model_directory/tokenizer_config.json',
 '/results/model_directory/special_tokens_map.json',
 '/results/model_directory/vocab.json',
 '/results/model_directory/merges.txt',
 '/results/model_directory/added_tokens.json',
 '/results/model_directory/tokenizer.json')

In [49]:
tokenizer=AutoTokenizer.from_pretrained('/results/model_directory')
model=AutoModelForSeq2SeqLM.from_pretrained('/results/model_directory')

/usr/local/lib/python3.11/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
